In [ ]:
using BenchmarkTools
using BoundingVolumeHierarchies
using LinearAlgebra
using StatsBase
using FileIO
import BoundingVolumeHierarchies: Segment, intersection
import GeometryTypes: HomogenousMesh
import BoundingVolumeHierarchies: PointT, _normal
using Meshes, MeshViz

In [ ]:
mesh1 = load("04.obj")
size(mesh1)

In [ ]:
max_x=-Inf
min_x= Inf
max_y=-Inf
min_y= Inf
max_z=-Inf
min_z= Inf

for i=1:size(mesh1)
    for j=1:3
        if mesh1[i][j][1]>max_x
            max_x=mesh1[i][j][1]
        end
        if mesh1[i][j][1]<min_x
            min_x=mesh1[i][j][1]
        end
        if mesh1[i][j][2]>max_y
             max_y=mesh1[i][j][2]
        end
        if mesh1[i][j][2]<min_y
            min_y=mesh1[i][j][2]
        end
        if mesh1[i][j][3]>max_z
            max_z=mesh1[i][j][3]
        end
        if mesh1[i][j][3]<min_z
            min_z=mesh1[i][j][3]
        end
    end
end
print((max_x,max_y,max_z))
print((min_x,min_y,min_z))

In [ ]:
start=Point3([min_x-0.1,min_y-0.1,min_z-0.1])
finish=Point3([max_x+0.1,max_y+0.1,max_z+0.1])
grid=CartesianGrid(start, finish, dims=(round(Int,(max_x-min_x)/0.2),round(Int,(max_y-min_y)/0.2),round(Int,(max_z-min_z)/0.2)))


In [ ]:
import WGLMakie as Mke
size1=size(grid)[1]*size(grid)[2]*size(grid)[3]
print(size1)
viz(grid, showfacets = true)

In [ ]:
Meshes.vertices(grid[1])[1]

In [ ]:
i=1
segm = Array{Segment}(undef, 12)
segm[1] = Segment(coordinates(Meshes.vertices(grid[i])[1]), coordinates(Meshes.vertices(grid[i])[2]))
segm[2] = Segment(coordinates(Meshes.vertices(grid[i])[2]), coordinates(Meshes.vertices(grid[i])[3]))
segm[3] = Segment(coordinates(Meshes.vertices(grid[i])[3]), coordinates(Meshes.vertices(grid[i])[4]))
segm[4] = Segment(coordinates(Meshes.vertices(grid[i])[4]), coordinates(Meshes.vertices(grid[i])[1]))
segm[5] = Segment(coordinates(Meshes.vertices(grid[i])[5]), coordinates(Meshes.vertices(grid[i])[6]))
segm[6] = Segment(coordinates(Meshes.vertices(grid[i])[6]), coordinates(Meshes.vertices(grid[i])[7]))
segm[7] = Segment(coordinates(Meshes.vertices(grid[i])[7]), coordinates(Meshes.vertices(grid[i])[8]))
segm[8] = Segment(coordinates(Meshes.vertices(grid[i])[8]), coordinates(Meshes.vertices(grid[i])[1]))
segm[9] = Segment(coordinates(Meshes.vertices(grid[i])[1]), coordinates(Meshes.vertices(grid[i])[5]))
segm[10] = Segment(coordinates(Meshes.vertices(grid[i])[2]), coordinates(Meshes.vertices(grid[i])[6]))
segm[11] = Segment(coordinates(Meshes.vertices(grid[i])[3]), coordinates(Meshes.vertices(grid[i])[7]))
segm[12] = Segment(coordinates(Meshes.vertices(grid[i])[4]), coordinates(Meshes.vertices(grid[i])[8]))

In [ ]:
#create point sets and face sets to build the BVH
rverts = [[0.0,0.0,0.0]]
rfaces = [[0,0,0]]
for i=1:5375
    for j=1:3
        push!(rverts,mesh1[i][j])
    end
    connec1=[[3i-2,3i-1,3i]]
    append!(rfaces,connec1)    
end

popfirst!(rfaces)
popfirst!(rverts)

#create BVH
normals = [_normal(rverts[f]) for f in rfaces] # mesh cell normals
rBVH = @time buildBVH(rverts, rfaces)

In [ ]:
#naive intersection
function intersection1(s::Segment, cell::Vector{Vector{Float64}}, cell_normal)
    dot(cell_normal, s.a-cell[1]) <= zero(Float32) && return false
    dot(cell_normal, s.b-cell[1]) >= zero(Float32) && return false
    BA = s.a - s.b
    dot(cross(cell[2]-cell[1], s.b-cell[2]), BA) < zero(Float32) && return false
    dot(cross(cell[3]-cell[2], s.b-cell[3]), BA) < zero(Float32) && return false
    dot(cross(cell[1]-cell[3], s.b-cell[1]), BA) < zero(Float32) && return false
    return true
end

function intersection2(s::Segment, bvh::BVH, vertices, faces, normals)
    for i=1:5375
        if(intersection1(s, vertices[faces[i]], normals[i]))
            return true
        end
    end
    return false
end

function intersection3(s::Segment, bvh::BVH, vertices, faces, normals)
    any(intersection1(s, vertices[faces[i]], normals[i]) for i=1:5375) && return true
    return false
end

In [ ]:
seg = Segment(Float32[-0.64122, 1.55488, 2.0093], Float32[-0.64122, 2.22515, 2.0093])

In [ ]:
#comparison of with(out)BVH
@time intersection(seg, rBVH, rverts, rfaces, normals)

In [ ]:
@time intersection3(seg, rBVH, rverts, rfaces, normals)

In [ ]:
global count=0
if(any(intersection(segm[i], rBVH, rverts, rfaces, normals) for i=1:12))
    count+=1
end

count

In [ ]:
Meshes.faces

In [ ]:
global segm = Array{Segment}(undef, 12)
global count=0
intersectionlist = Any[]
for i=1:size1
    segm[1] = Segment(coordinates(Meshes.vertices(grid[i])[1]), coordinates(Meshes.vertices(grid[i])[2]))
    segm[2] = Segment(coordinates(Meshes.vertices(grid[i])[2]), coordinates(Meshes.vertices(grid[i])[3]))
    segm[3] = Segment(coordinates(Meshes.vertices(grid[i])[3]), coordinates(Meshes.vertices(grid[i])[4]))
    segm[4] = Segment(coordinates(Meshes.vertices(grid[i])[4]), coordinates(Meshes.vertices(grid[i])[1]))
    segm[5] = Segment(coordinates(Meshes.vertices(grid[i])[5]), coordinates(Meshes.vertices(grid[i])[6]))
    segm[6] = Segment(coordinates(Meshes.vertices(grid[i])[6]), coordinates(Meshes.vertices(grid[i])[7]))
    segm[7] = Segment(coordinates(Meshes.vertices(grid[i])[7]), coordinates(Meshes.vertices(grid[i])[8]))
    segm[8] = Segment(coordinates(Meshes.vertices(grid[i])[8]), coordinates(Meshes.vertices(grid[i])[1]))
    segm[9] = Segment(coordinates(Meshes.vertices(grid[i])[1]), coordinates(Meshes.vertices(grid[i])[5]))
    segm[10] = Segment(coordinates(Meshes.vertices(grid[i])[2]), coordinates(Meshes.vertices(grid[i])[6]))
    segm[11] = Segment(coordinates(Meshes.vertices(grid[i])[3]), coordinates(Meshes.vertices(grid[i])[7]))
    segm[12] = Segment(coordinates(Meshes.vertices(grid[i])[4]), coordinates(Meshes.vertices(grid[i])[8]))
    if(any(intersection(segm[i], rBVH, rverts, rfaces, normals) for i=1:12))
        count+=1
        push!(intersectionlist,i)
    end
end
    
    
    

In [ ]:
length(intersectionlist)

In [ ]:
IntersectionType

In [ ]:
f=viz(grid[intersectionlist[1]])
for i=2:length(intersectionlist)
    viz!(grid[intersectionlist[i]])
end

In [ ]:
display(f)

In [ ]:
#create point sets and connection sets to form the mesh
points = Point3[]
connec = connect.([(0,0,0)], Ngon)

for i=1:5375
    for j=1:3
        push!(points,Point3(mesh1[i][j]))
    end
    A=(3i-2,3i-1,3i)
    connect.([A],Ngon)
    append!(connec,connect.([A],Ngon))    
end

popfirst!(connec)

# the processed mesh
processed_mesh = SimpleMesh(points, connec)

viz(processed_mesh, showfacets = true)

In [ ]:
#currently unsuccessful plotting of BVH
using Makie

## Plot the mesh together with the bounding boxes in the Bounding Volume Hierarchy

scene = Scene(center=false)

wireframe!(scene, mesh1, linewidth=0.5, color=:black)

for bv in boundingvolumes(rBVH)
    wireframe!(scene, HomogenousMesh(vertices(bv), faces(bv)), linewidth = 0.5, color=:red)
end

cam = Makie.cameracontrols(scene)
cam.upvector[] = Point(0.,1.,0.)
cam.eyeposition[] = Point(3,2.,4)
update_cam!(scene, cam)

display(scene)


## benchmark naive method vs using a BVH

# segments from a fixed source to a random point constrained to a plane
# such that the mesh is between the source and the plane
create_segments() =  [Segment(PointT(1, 0.4, 0.4),
                              PointT(-0.4, rand(), 2*rand().-0.7)) for _ in 1:10_000]

segments = create_segments()

# check all meshcell for each segment
naiveintersection(s, verts, faces, normals) = any(intersection(s, verts[f], n) for (f,n) in zip(faces, normals))

naive_hits = @btime [naiveintersection(s, rverts, rfaces, rnormals) for s in segments]
bvh_hits = @btime [intersection(s, rBVH, rverts, rfaces, rnormals) for s in segments]
@assert all(bvh_hits .== naive_hits)


##

# Plot the mesh together with a scatter of the end points of a set of segments
# all segments starts at the same center point (large orange ball)
# all segments end on a plane on the opposite side of the cat mesh,
# end ponints are colored black if the segment intersects with the catmesh
# and orange otherwise.

scene = Scene(center=false)

wireframe!(scene, mesh1, linewidth=0.5, color=:black)
scatter!(scene, [s.b for (s,h) in zip(segments, bvh_hits) if h], color=:black, markersize=0.01)
scatter!(scene, [s.b for (s,h) in zip(segments, bvh_hits) if !h], color=:orange, markersize=0.01)
scatter!(scene, [segments[1].a], color=:orange)

cam = Makie.cameracontrols(scene)
cam.upvector[] = Point(0.,1.,0.)
cam.eyeposition[] = Point(3,2.,4)
update_cam!(scene, cam)

display(scene)